# Notebook for API Financials Importing

# 0. Preamble

In [46]:
# Import yahoo finance API and pandas
import yfinance as yf
import pandas as pd
import random
import datetime
import numpy as np

## Ticker List - Keep Closed lol

In [47]:
"""
Largest 100 companies by market capitalisation (source: GPT-o1). If a ticker did not contain all 4 vars (net_income, net_assets, current_ratio, free_cash_flow),
Pas dropped the ticker and appended the 100+nth largest company's ticker. n indexes the companies with incomplete data.
"""

largest_100_tickers = [
    "AAPL",      # Apple
    "MSFT",      # Microsoft
    "GOOGL",     # Alphabet Class A
    "AMZN",      # Amazon
    "NVDA",      # NVIDIA
    "TSLA",      # Tesla
    "META",      # Meta Platforms
    #"BRK.B",     # Berkshire Hathaway (Class B)    # Warning: Missing key data for BRK.B: 'Net Income'
    "TSM",       # Taiwan Semiconductor
    "V",         # Visa
    "XOM",       # Exxon Mobil
    "UNH",       # UnitedHealth
    "JNJ",       # Johnson & Johnson
    # "JPM",       # JPMorgan Chase   # Warning: Missing key data for JPM: 'Current Assets'
    "WMT",       # Walmart
    "PG",        # Procter & Gamble
    "HD",        # Home Depot
    "MA",        # Mastercard
    "CVX",       # Chevron
    "LLY",       # Eli Lilly
    "PEP",       # PepsiCo
    "NSRGY",     # Nestlé (ADR)
    "ABBV",      # AbbVie
    "MRK",       # Merck & Co.
    "RHHBY",     # Roche Holding (ADR)
    "KO",        # Coca-Cola
    "LVMUY",     # LVMH (ADR)
    # "BAC",       # Bank of America    # Warning: Missing key data for BAC: 'Current Assets'
    "NVO",       # Novo Nordisk (ADR)
    "PFE",       # Pfizer
    "AVGO",      # Broadcom
    "CMCSA",     # Comcast
    "TMO",       # Thermo Fisher Scientific
    "NKE",       # Nike
    "CRM",       # Salesforce
    "ADBE",      # Adobe
    "NFLX",      # Netflix
    "TM",        # Toyota Motor (ADR)
    "CSCO",      # Cisco
    "AZN",       # AstraZeneca
    "ACN",       # Accenture
    "SHEL",      # Shell (ADR)
    "INTC",      # Intel
    "MCD",       # McDonald's
    "VZ",        # Verizon
    "600519.SS", # Kweichow Moutai (Shanghai)
    "DIS",       # Walt Disney
    "BUD",       # AB InBev (ADR)
    "SAP",       # SAP SE
    "TMUS",      # T-Mobile US
    "ORCL",      # Oracle
    "LRLCY",     # L'Oréal (ADR)
    # "SNP",       # Sinopec (ADR)    # Warning: Missing key data for SNP: 'Net Income'
    "ASML",      # ASML Holding
    "AMT",       # American Tower
    "COP",       # ConocoPhillips
    "PM",        # Philip Morris International
    "DHR",       # Danaher
    "UNP",       # Union Pacific
    "TXN",       # Texas Instruments
    "IBM",       # IBM
    "RTX",       # Raytheon Technologies
    "COST",      # Costco
    # "CICHY",     # China Construction Bank (ADR)    # Warning: Missing key data for CICHY: 'Current Assets'
    "SHOP",      # Shopify
    # "HSBC",      # HSBC Holdings    # Warning: Missing key data for HSBC: 'Current Assets'
    "BMY",       # Bristol Myers Squibb
    # "ACGBY",     # Agricultural Bank of China (ADR)   # Warning: Missing key data for ACGBY: 'Current Assets'
    # "WFC",       # Wells Fargo     # Warning: Missing key data for WFC: 'Current Assets'
    "BHP",       # BHP Group
    # "RY",        # Royal Bank of Canada   # Warning: Missing key data for RY: 'Current Assets'
    # "C",         # Citigroup   # Warning: Missing key data for C: 'Current Assets'
    "NEE",       # NextEra Energy
    # "CHL",       # China Mobile (ADR)      # Warning: Missing key data for CHL: 'Net Income'
    # "PTR",       # PetroChina (ADR)   # Warning: Missing key data for PTR: 'Net Income'
    "MO",        # Altria
    "GSK",       # GSK (GlaxoSmithKline)
    "TTE",       # TotalEnergies
    "BABA",      # Alibaba
    "MDLZ",      # Mondelez
    "T",         # AT&T
    "DUK",       # Duke Energy
    "SBUX",      # Starbucks
    "SONY",      # Sony (ADR)
    "PBR",       # Petrobras (ADR)
    "INTU",      # Intuit
    "AMD",       # Advanced Micro Devices
    "DE",        # Deere & Company
    # "GS",        # Goldman Sachs       # Warning: Missing key data for GS: 'Current Assets'
    "BLK",       # BlackRock
    "ADP",       # Automatic Data Processing
    "GILD",      # Gilead Sciences
    "LOW",       # Lowe's
    "QCOM",      # Qualcomm
    "UPS",       # United Parcel Service
    "PLD",       # Prologis
    "BCE",       # BCE Inc.
    "TGT",       # Target
    "MMC",       # Marsh & McLennan
    # "SCHW"       # Charles Schwab      # Warning: Missing key data for SCHW: 'Current Assets'
    "BA",   # Boeing
    "CAT",  # Caterpillar
    # "AXP",  # American Express      # Warning: Missing key data for AXP: 'Current Assets'
    # "MS",   # Morgan Stanley      # Warning: Missing key data for MS: 'Current Assets'
    "UL",   # Unilever
    "RIO",  # Rio Tinto
    "BP",   # BP plc
    "FDX",  # FedEx
    "CVS",  # CVS Health
    "SNY",  # Sanofi
    # "PNC",  # PNC Financial      # Warning: Missing key data for PNC: 'Current Assets'
    "ENB",  # Enbridge
    "MU",   # Micron Technology
    # "BK"    # Bank of New York Mellon      # Warning: Missing key data for BK: 'Current Assets'
    "ZTS",  # Zoetis
    "PAYX", # Paychex
    "CSX",  # CSX Corp
    "GM"    # General Motors
]

## 1. API Call Function

In [48]:
def get_financial_data(tickers, start_date="2023-01-01", end_date="2024-01-01"):
    """
    Fetch Net Income, Net Assets, Current Ratio, and Free Cash Flow
    for a given list of tickers using yfinance.
    Returns a pandas DataFrame with the results.
    """

    # Initialise list for financial records
    records = []

    # Loop over tickers (with print verifications)
    for ticker_symbol in tickers:
        print(f"Fetching data for {ticker_symbol}...")
        ticker = yf.Ticker(ticker_symbol)

        # 1. Financial Statements Data - all relevant variables from the "Financial Statement" section of the yahoo finance API.
        try:
            income_stmt = ticker.financials
            balance_sheet = ticker.balance_sheet
            cash_flow = ticker.cashflow

            net_income = income_stmt.loc["Net Income"].iloc[0]
            total_assets = balance_sheet.loc["Total Assets"].iloc[0]
            total_liabilities = balance_sheet.loc["Total Liabilities Net Minority Interest"].iloc[0]
            net_assets = total_assets - total_liabilities

            total_current_assets = balance_sheet.loc["Current Assets"].iloc[0]
            total_current_liabilities = balance_sheet.loc["Current Liabilities"].iloc[0]
            current_ratio = total_current_assets / total_current_liabilities

            free_cash_flow = cash_flow.loc["Free Cash Flow"].iloc[0]

        # If yahoo finance does not contain the specified variable name an exception is raised on the missing variable (Key) "e"
        # The stated variables are then set to "None"
        except KeyError as e:
            print(f"Warning: Missing key data for {ticker_symbol}: {e}")
            net_income = None
            net_assets = None
            current_ratio = None
            free_cash_flow = None

        # 2. Historical Stock Prices --> Yearly Average Close
        try:
            # Fetch daily historical data
            df_price = ticker.history(start=start_date, end=end_date)

            if df_price.empty:
                # If no data returned, store None or an empty dict
                yearly_avg_close = {}
            else:
                # Extract the year from the DatetimeIndex
                df_price["Year"] = df_price.index.year

                # Group by year and compute mean of 'Close'
                yearly_avg_series = df_price.groupby("Year")["Close"].mean()

                # Convert the resulting Series to a dictionary: {year: avg_close, ...}
                yearly_avg_close = yearly_avg_series.to_dict()

        except Exception as e:
            print(f"Warning: Could not retrieve price data for {ticker_symbol}: {e}")
            yearly_avg_close = {}

        # 3. Store everything in one record
        record = {
            "Ticker": ticker_symbol,
            "Net Income": net_income,
            "Net Assets": net_assets,
            "Current Ratio": current_ratio,
            "Free Cash Flow": free_cash_flow,
            "Yearly Average Close": yearly_avg_close  # dictionary of {year: avg_close}
        }

        records.append(record)

    df_results = pd.DataFrame(records)
    return df_results

# 2. Execute Main Function

In [49]:
if __name__ == "__main__":
    # Example usage:
    my_tickers = largest_100_tickers
    df = get_financial_data(my_tickers,
                            start_date="2023-01-01",
                            end_date="2024-01-01")
    # print("Original DataFrame:\n", df, "\n")

    # 1) Set Ticker as index, so columns become the data fields
    df.set_index("Ticker", inplace=True)

    # 2) Transpose, so now rows = features, columns = tickers
    df_transposed = df.T
    print("Transposed DataFrame (features x entities):\n", df_transposed, "\n")


## Training-test split
    # df_transposed.columns are entities/tickers
    test_size = 20

    # Randomly choose 'sample_size' columns
    selected_columns = random.sample(df_transposed.columns.tolist(), k=test_size)

    # Create df_test (test) with only sampled columns
    df_test = df_transposed[selected_columns]

    # Create df_train (train) from the unsampled columns
    unsampled_columns = [col for col in df_transposed.columns if col not in selected_columns]
    df_train = df_transposed[unsampled_columns]

## Input-output split
    selected_rows = ["Yearly Average Close"]
    df_y_test = df_test.loc[selected_rows]
    df_y_train = df_train.loc[selected_rows]

    ## Extracting numerals from y data
    def extract_numeric(d):
        # Assumes each dict has a single key-value pair
        return list(d.values())[0]

    # Apply extraction to the entire DataFrame
    df_y_test = df_y_test.map(extract_numeric)
    df_y_train = df_y_train.map(extract_numeric)

    df_y_test.to_csv("y_test.csv")
    df_y_train.to_csv("y_train.csv")

    print("df_y_test (test) shape:", df_y_test.shape)
    print("df_y_train (train) shape:", df_y_train.shape)

    df_x_test = df_test.drop(selected_rows, axis=0)
    df_x_train = df_train.drop(selected_rows, axis=0)

    df_x_test.to_csv("x_test.csv")
    df_x_train.to_csv("x_train.csv")


## Make Arrays
    x_test = df_x_test.to_numpy()
    x_train = df_x_train.to_numpy()

    y_test = df_y_test.to_numpy()
    y_train = df_y_train.to_numpy()

    print("df_transposed shape:", df_transposed.shape)

    # print x arrays
    print("df_x_test (test) shape:", df_x_test.shape)
    print("df_x_train (train) shape:", df_x_train.shape)

    # print y arrays
    print("df_y_test (test) shape:", df_y_test.shape)
    print("df_y_train (train) shape:", df_y_train.shape)

Fetching data for AAPL...
Fetching data for MSFT...
Fetching data for GOOGL...
Fetching data for AMZN...
Fetching data for NVDA...
Fetching data for TSLA...
Fetching data for META...
Fetching data for TSM...
Fetching data for V...
Fetching data for XOM...
Fetching data for UNH...
Fetching data for JNJ...
Fetching data for WMT...
Fetching data for PG...
Fetching data for HD...
Fetching data for MA...
Fetching data for CVX...
Fetching data for LLY...
Fetching data for PEP...
Fetching data for NSRGY...
Fetching data for ABBV...
Fetching data for MRK...
Fetching data for RHHBY...
Fetching data for KO...
Fetching data for LVMUY...
Fetching data for NVO...
Fetching data for PFE...
Fetching data for AVGO...
Fetching data for CMCSA...
Fetching data for TMO...
Fetching data for NKE...
Fetching data for CRM...
Fetching data for ADBE...
Fetching data for NFLX...
Fetching data for TM...
Fetching data for CSCO...
Fetching data for AZN...
Fetching data for ACN...
Fetching data for SHEL...
Fetching d

<ipython-input-49-b8f77b066b2e>:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_y_test = df_y_test.applymap(extract_numeric)
<ipython-input-49-b8f77b066b2e>:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_y_train = df_y_train.applymap(extract_numeric)


## Data Examination

In [50]:
print(type(y_test), y_test)

<class 'numpy.ndarray'> [[ 58.11425084 293.80613513 527.59230615 212.07215997 143.52608508
  269.08994098  70.22498384  50.04365074  58.62220111 103.6952254
  202.86212952  33.91745524  65.53633621 140.46356641 221.87278107
   34.27049236 154.18118793 103.02935989 159.50707953  86.03647522]]


AttributeError: 'numpy.float64' object has no attribute 'values'

## Test Section

In [ ]:
import yfinance as yf
import pandas as pd

# Show all rows
pd.set_option('display.max_rows', None)
# Show all columns
pd.set_option('display.max_columns', None)

ticker = yf.Ticker("AAPL")  # or any other ticker
balance_sheet = ticker.balance_sheet

print(balance_sheet)               # Print the whole DataFrame
print(balance_sheet.index.tolist())  # Print just the row labels